**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
! pip install scikit-video

     |████████████████████████████████| 2.3MB 22.6MB/s 


In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

import matplotlib.pyplot as plt
from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act is used to return the next action to do. We have to distinguish two different cases : 
* if the agent is training : at random we decide if we take a random decision ($ p= \epsilon $) or if we want to apply the current learned policy ($ p= 1 - \epsilon $). 
* if the agent is not training: we directly apply the learned policy

The ```epsilon``` parameter is used to fix the probability $\epsilon$ of taking a random decision instead of applying the learned policy. The higher the ``` epsilon``` is, the more the agent is prone to exploration. The optimisation problem being non-convex, it is important to explore new strategy by taking a first step in a direction that is not locally optimal otherwise the learning will converge to a local optimum. Reducing the value of ```epsilon```  increases the exploitation of previous knowledge, making the training more likely to converge. Therefore, setting the ```epsilon``` parameter requires to find the good trade off between the quality of the solution learned and the stability of the training. 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=25 # set small when debugging
epochs_test=15 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The arrays ```position```  and ```board``` are both array of the size of the grid. 

* The ```position``` array stores the position of the rat which is the only element with a ```1```, others are ```0``` (or ```-1``` on the edges).  
* The ```board``` array is used to store the information about the cells of the game : a poisonous cell has a value of -1 and a cheesy cell has a value of 0.5. Once the rat has been on a cell, the  value is changed to 0 (the content of the cell has been eatend by the rat). 

Both arrays are used as their concatenation is the state of the game. 

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix='', verbose=2):
    # Number of won games
    score = 0

    for e in range(epochs):

        state = env.reset()
        game_over = False
        win = 0
        lose = 0
        while not game_over:
          # The agent performs an action
          action = agent.act(state)

          # Apply an action to the environment, get the next state, the reward
          # and if the games end
          prev_state = state
          state, reward, game_over = env.act(action)

          # Update the counters
          if reward > 0:
              win = win + reward
          if reward < 0:
              lose = lose -reward
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose
        if verbose > 1 : 
             print("Win/lose count {}/{}. Average score ({})"
                   .format(win, lose, score/(1+e)))
    if verbose > 0 : print('Final score: '+str(score/epochs))
    return(score)

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 12.0/17.0. Average score (-5.0)
Win/lose count 8.5/16.0. Average score (-6.25)
Win/lose count 8.5/12.0. Average score (-5.333333333333333)
Win/lose count 6.0/13.0. Average score (-5.75)
Win/lose count 12.0/10.0. Average score (-4.2)
Win/lose count 12.5/23.0. Average score (-5.25)
Win/lose count 5.5/19.0. Average score (-6.428571428571429)
Win/lose count 11.0/18.0. Average score (-6.5)
Win/lose count 8.0/13.0. Average score (-6.333333333333333)
Win/lose count 11.0/15.0. Average score (-6.1)
Win/lose count 8.5/8.0. Average score (-5.5)
Win/lose count 12.5/10.0. Average score (-4.833333333333333)
Win/lose count 13.5/16.0. Average score (-4.653846153846154)
Win/lose count 9.5/14.0. Average score (-4.642857142857143)
Win/lose count 10.0/14.0. Average score (-4.6)
Final score: -4.6


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




1. The definition of $ Q^{\pi}(s, a) $ gives us  : 
$$ Q^{\pi}(s, a) = E_{p^{\pi}} \left[ \sum_{t \leq T} \gamma^t r(s_t, a_t) | s_0=s, a_0=a \right] $$
Separating the term of index $0$ form the sum, we have by linearity of the expectation :
$$ Q^{\pi}(s, a) = E_{p^{\pi}} \left[ r(s_0, a_0) | s_0=s, a_0=a \right] + E_{p^{\pi}} \left[ \sum_{1 < t \leq T} \gamma^t r(s_t, a_t) | s_0=s, a_0=a \right] $$
Since, $$ E_{p^{\pi}} \left[ r(s_0, a_0) | s_0=s, a_0=a \right] = r(s,a)$$ 
it follows that : 
$$ Q^{\pi}(s, a) = r(s, a) + E_{p^{\pi}} \left[ \gamma  \sum_{1 < t \leq T} \gamma^{t-1} r(s_t, a_t) | s_0=s, a_0=a \right]$$
We have : 
$$ E_{p^{\pi}} \left[\sum_{1 < t \leq T} \gamma^{t-1} r(s_t, a_t) | s_0=s, a_0=a \right] = E_{p^{\pi}} \left[ E_{p^{\pi}} \left[ \sum_{1 \leq t \leq T} \gamma^{t-1} r(s_t, a_t) | s_1=s', a_1=a' \right] | s_0=s, a_0=a \right]$$ 
Thus,
$$ E_{p^{\pi}} \left[\sum_{1 < t \leq T} \gamma^{t-1} r(s_t, a_t) | s_0=s, a_0=a \right]  = E_{(s', a') \sim p^{\pi}(\cdot|s,a)} \left[ E_{p^{\pi}} \left[ \sum_{0 \leq t \leq T-1} \gamma^{t} r(s_t, a_t) | s_0=s', a_0=a' \right] \right] $$
which gives, 
$$ Q^{\pi}(s, a) = r(s, a) + \gamma E_{(s', a') \sim p^{\pi}(\cdot|s,a)} \left[ E_{p^{\pi}} \left[ \sum_{0 \leq t \leq T-1} \gamma^{t} r(s_t, a_t) | s_0=s', a_0=a' \right] \right]$$
It follows directly:
$$Q^{\pi}(s, a) = E_{(s', a') \sim p^{\pi}(\cdot|s,a)} \left[ r(s, a) + \gamma Q^{\pi}(s', a') \right]$$

2. From the previous point, we have : 
$$ Q^{*}(s,a) = \max_{\pi} E_{(s',a')\sim p^{\pi}(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] $$ Let's consider $\pi ^ *$, an optimal policy (we assume its existence). It follows: $$ Q^{*}(s,a) = E_{(s',a')\sim p^{\pi^ * }(.|s,a)}[r(s,a)+\gamma Q^{\pi ^ *}(s',a')] $$
$\pi^*$ being optimal for $a'$ the action taken following $\pi^*$ $$ Q^{\pi ^ *}(s',a') = \max_{b}Q^*(s',b) $$ The policy being fixed (and deterministic) : 
$$ Q^{*}(s,a) = E_{s'\sim p(.|s,a)}[r(s,a)+\gamma \max_{b}Q^*(s',b)] $$ 


3. The previous point can be written : $$  E_{s'\sim p(.|s,a)}[r(s,a)+\gamma \max_{b}Q^*(s',b) - Q^{*}(s,a)] = 0 $$ We try to estimate $Q^*$ with $Q(.,.|\theta)$ so the minimizing the given objective is equivalent to minimizing the norm of the estimate of the previous term. Therefore it is a plausible objective. 


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) == self.max_memory:
            self.memory = self.memory[1:]
        self.memory.append(m)

    def random_access(self):
        index = np.random.randint(0, len(self.memory))
        return self.memory[index]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix='', verbose=1):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        if verbose > 0:
          print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
                .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.array([s]))[0])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        batch_size = min(len(self.memory.memory), self.batch_size)

        minibatch = [self.memory.random_access() for i in range(self.batch_size)]

        previous_states = np.array([values[0] for values in minibatch])
        updated_previous_Qs = self.model.predict(previous_states)

        states = np.array([values[1] for values in minibatch])
        updated_Qs = self.model.predict(states)

        target_q = np.zeros((self.batch_size, 4))

        for i, (current_state, new_current_state, action, reward, game_over) in enumerate(minibatch):
            if game_over:
                y_j = reward
            else:
                y_j = reward + self.discount * np.max(updated_Qs[i])
              
            target_q[i] = updated_Qs[i]
            target_q[i, action] = y_j

        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(previous_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, n_fc = 2, dense_cells = 20, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        ######
        input_model = keras.layers.Input(shape=(5, 5, self.n_state), name="input")
        x = keras.layers.Flatten(name="flatten")(input_model)

        for i in range(n_fc):
            x = Dense(dense_cells, name="fc{}".format(i), activation="relu")(x)
        output_model = Dense(4, name="output")(x)
        model = keras.models.Model(input_model, output_model)
        ########
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

### Testing the FC agent

In [53]:
epochs_train = 25
epochs_test = 10
lr = 0.1
epsilon = 0.1
memory_size = 2000
batch_size = 32
dense_cells = 10
n_fc = 1

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=lr, epsilon = epsilon, memory_size=memory_size, batch_size = batch_size)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

print('Test of the FC')
scores = []
for i in range(epochs_test):
    scores.append(test(agent, env, 1, prefix='fc_test', verbose = 0))
print('Average Score',  np.mean(scores))
print('score standard deviation',  np.std(scores))

HTML(display_videos('fc_test0.mp4'))

Epoch 000/025 | Loss 0.0245 | Win/lose count 3.0/3.0 (0.0)
Epoch 001/025 | Loss 0.0265 | Win/lose count 5.0/3.0 (2.0)
Epoch 002/025 | Loss 0.0203 | Win/lose count 2.5/3.0 (-0.5)
Epoch 003/025 | Loss 0.0085 | Win/lose count 4.0/6.0 (-2.0)
Epoch 004/025 | Loss 0.0219 | Win/lose count 3.5/6.0 (-2.5)
Epoch 005/025 | Loss 0.0215 | Win/lose count 3.5/6.0 (-2.5)
Epoch 006/025 | Loss 0.0157 | Win/lose count 6.5/8.0 (-1.5)
Epoch 007/025 | Loss 0.0054 | Win/lose count 5.0/4.0 (1.0)
Epoch 008/025 | Loss 0.0058 | Win/lose count 4.0/10.0 (-6.0)
Epoch 009/025 | Loss 0.0126 | Win/lose count 5.5/6.0 (-0.5)
Epoch 010/025 | Loss 0.0142 | Win/lose count 9.0/7.0 (2.0)
Epoch 011/025 | Loss 0.0061 | Win/lose count 2.5/4.0 (-1.5)
Epoch 012/025 | Loss 0.0180 | Win/lose count 2.5/3.0 (-0.5)
Epoch 013/025 | Loss 0.0182 | Win/lose count 4.0/3.0 (1.0)
Epoch 014/025 | Loss 0.0096 | Win/lose count 10.0/6.0 (4.0)
Epoch 015/025 | Loss 0.0218 | Win/lose count 7.0/11.0 (-4.0)
Epoch 016/025 | Loss 0.0229 | Win/lose coun

### Finding a good set of hyperparameters for the FC agent

Given the high variance of the results between the experiments with same hyperparameters, we won't spend too much time on the search of a good set of hyperparameters.  

For this we will use a very basic random search and retiterate with a smaller set carefully chosen. 

In [74]:
epochs_train = 75
epochs_test = 30

dense_cells = 50
n_fc = 3

memory_size = 2000
batch_size = 32


n_fcs = [2 ,3]
dense_cells = [32, 64]
epsilons = [0.2]
lrs = [0.5]

res  =  {}
for n_fc in n_fcs:
    res[str(n_fc)] = {}
    for dense_cell in dense_cells: 
        res[str(n_fc)][str(dense_cell)]= {}
        for epsilon in epsilons:
            res[str(n_fc)][str(dense_cell)][str(epsilon)] = {}
            for lr in lrs :
                env = Environment(grid_size=size, max_time=T, temperature=0.3)
                agent = DQN_FC(size, n_fc = n_fc, dense_cells = dense_cell, lr=lr, epsilon = epsilon, memory_size=memory_size, batch_size = batch_size)
                train(agent, env, epochs_train, prefix='fc_train', verbose = 0)
                HTML(display_videos('fc_train10.mp4'))

                agent_fc = DQN_FC(size, n_fc = n_fc, dense_cells = dense_cell, lr=lr, epsilon = epsilon, memory_size=memory_size, batch_size = batch_size)
                agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
                env = Environment(grid_size=size, max_time=T, temperature=0.3)
                print('Test of the FC')
                scores = []
                for i in range(epochs_test):
                    scores.append(test(agent_fc, env, 1, prefix='fc_test', verbose = 0))
                res[str(n_fc)][str(dense_cell)][str(epsilon)][str(lr)] = 'Average Score: {} | score standard deviation : {}'.format(np.mean(scores), np.std(scores))
                print('Scores for : nfc : {} | dense_cell : {} | epsilon : {} | lr : {}'.format(n_fc, dense_cell, epsilon, lr))
                print('Average Score',  np.mean(scores))
                print('score standard deviation',  np.std(scores))


Test of the FC
Scores for : nfc : 2 | dense_cell : 32 | epsilon : 0.2 | lr : 0.5
Average Score 2.2666666666666666
score standard deviation 2.7041120949809425
Test of the FC
Scores for : nfc : 2 | dense_cell : 64 | epsilon : 0.2 | lr : 0.5
Average Score 7.783333333333333
score standard deviation 3.9658612627720045
Test of the FC
Scores for : nfc : 3 | dense_cell : 32 | epsilon : 0.2 | lr : 0.5
Average Score 7.65
score standard deviation 4.070524126776141
Test of the FC
Scores for : nfc : 3 | dense_cell : 64 | epsilon : 0.2 | lr : 0.5
Average Score 3.9
score standard deviation 3.0534133905079623


In [61]:
epochs_train = 75
epochs_test = 10
lr = 0.5
epsilon = 0.2
memory_size = 2000
batch_size = 32
dense_cells = 64
n_fc = 2

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=lr, epsilon = epsilon, memory_size=memory_size, batch_size = batch_size)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

print('Test of the FC')
scores = []
for i in range(epochs_test):
    scores.append(test(agent, env, 1, prefix='fc_test', verbose = 0))
print('Average Score',  np.mean(scores))
print('score standard deviation',  np.std(scores))

HTML(display_videos('fc_test0.mp4'))

Epoch 000/075 | Loss 0.0141 | Win/lose count 5.5/6.0 (-0.5)
Epoch 001/075 | Loss 0.0045 | Win/lose count 6.0/6.0 (0.0)
Epoch 002/075 | Loss 0.0084 | Win/lose count 1.0/5.0 (-4.0)
Epoch 003/075 | Loss 0.0629 | Win/lose count 2.5/2.0 (0.5)
Epoch 004/075 | Loss 0.0056 | Win/lose count 7.5/7.0 (0.5)
Epoch 005/075 | Loss 0.0033 | Win/lose count 7.5/6.0 (1.5)
Epoch 006/075 | Loss 0.0077 | Win/lose count 4.0/1.0 (3.0)
Epoch 007/075 | Loss 0.0106 | Win/lose count 7.0/7.0 (0.0)
Epoch 008/075 | Loss 0.0187 | Win/lose count 9.0/7.0 (2.0)
Epoch 009/075 | Loss 0.0118 | Win/lose count 9.5/8.0 (1.5)
Epoch 010/075 | Loss 0.0130 | Win/lose count 8.5/8.0 (0.5)
Epoch 011/075 | Loss 0.0099 | Win/lose count 7.5/7.0 (0.5)
Epoch 012/075 | Loss 0.0076 | Win/lose count 3.0/5.0 (-2.0)
Epoch 013/075 | Loss 0.0240 | Win/lose count 5.5/9.0 (-3.5)
Epoch 014/075 | Loss 0.0115 | Win/lose count 8.5/4.0 (4.5)
Epoch 015/075 | Loss 0.0099 | Win/lose count 7.0/6.0 (1.0)
Epoch 016/075 | Loss 0.0043 | Win/lose count 10.0/8.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,n_blocks=2, filter_base=32, lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        input_model = keras.layers.Input(shape=(5, 5, self.n_state), name="input")
        x = input_model
        for n_block in range(n_blocks):
            x = Conv2D((2**n_block) * filter_base, 3, padding="same", activation="relu", name="conv{}_1".format(n_block))(x)
            x = Conv2D((2**n_block) * filter_base, 3, padding="same", activation="relu", name="conv{}_2".format(n_block))(x)
            x = MaxPooling2D(name="pooled_{}".format(n_block), strides=1)(x) # we are not downscaling 
        x = keras.layers.Flatten(name="flatten")(x)
        x = Dense(4, name="output")(x)
        model = keras.models.Model(input_model, x)

        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model


### Testing the CNN agent

In [0]:
epochs_train = 50
epochs_test = 10

memory_size = 2000
batch_size = 32


n_block = 2
filter_base = 64
epsilon = 0.1
lr = 0.1

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(n_blocks=n_block, filter_base=filter_base, lr=lr, grid_size=size, epsilon=epsilon, memory_size=memory_size, batch_size=batch_size)
train(agent, env, epochs_train, prefix='cnn_train')
HTML(display_videos('cnn_train20.mp4'))

print('Test of the CNN')
test(agent, env, epochs_test, prefix='cnn_test')

Epoch 000/050 | Loss 0.0273 | Win/lose count 5.5/4.0 (1.5)
Epoch 001/050 | Loss 0.0073 | Win/lose count 2.5/7.0 (-4.5)
Epoch 002/050 | Loss 0.0179 | Win/lose count 11.0/15.0 (-4.0)
Epoch 003/050 | Loss 0.0166 | Win/lose count 15.5/13.0 (2.5)
Epoch 004/050 | Loss 0.0071 | Win/lose count 5.5/4.0 (1.5)
Epoch 005/050 | Loss 0.0061 | Win/lose count 7.5/3.0 (4.5)
Epoch 006/050 | Loss 0.0102 | Win/lose count 8.0/6.0 (2.0)
Epoch 007/050 | Loss 0.0136 | Win/lose count 6.5/6.0 (0.5)
Epoch 008/050 | Loss 0.0146 | Win/lose count 5.5/2.0 (3.5)
Epoch 009/050 | Loss 0.0055 | Win/lose count 11.5/3.0 (8.5)
Epoch 010/050 | Loss 0.0085 | Win/lose count 13.0/5.0 (8.0)
Epoch 011/050 | Loss 0.0055 | Win/lose count 8.5/5.0 (3.5)
Epoch 012/050 | Loss 0.0148 | Win/lose count 10.0/7.0 (3.0)
Epoch 013/050 | Loss 0.0022 | Win/lose count 17.5/4.0 (13.5)
Epoch 014/050 | Loss 0.0702 | Win/lose count 6.5/4.0 (2.5)
Epoch 015/050 | Loss 0.0177 | Win/lose count 11.5/4.0 (7.5)
Epoch 016/050 | Loss 0.0096 | Win/lose count

34.5

### Finding a good set of hyperparameters

The approach is the same as for the FC agent

In [0]:
epochs_train = 50
epochs_test = 10

dense_cells = 50
n_fc = 3

memory_size = 2000
batch_size = 32


n_blocks = [1, 2 ,3]
filter_bases = [32, 64]
epsilons = [0.1, 0.2]
lrs = [0.1, 0.5]

res  =  {}
for n_block in n_blocks:
    res[str(n_block)] = {}
    for filter_base in filter_bases: 
        res[str(n_block)][str(filter_base)]= {}
        for epsilon in epsilons:
            res[str(n_block)][str(filter_base)][str(epsilon)] = {}
            for lr in lrs :
                env = Environment(grid_size=size, max_time=T, temperature=0.3)
                agent = DQN_CNN(size, n_blocks=n_block, filter_base = filter_base, lr=lr, epsilon = epsilon, memory_size=memory_size, batch_size = batch_size)
                train(agent, env, epochs_train, prefix='CNN_train', verbose = 0)
                print('Test of the CNN')
                scores = []
                for i in range(epochs_test):
                    scores.append(test(agent, env, 1, prefix='CNN_test', verbose = 0))
                res[str(n_block)][str(filter_base)][str(epsilon)][str(lr)] = 'Average Score: {} | score standard deviation : {}'.format(np.mean(scores), np.std(scores))
                print('Scores for : n_blocks : {} | filter_base : {} | epsilon : {} | lr : {}'.format(n_block, filter_base, epsilon, lr))
                print('Average Score',  np.mean(scores))
                print('score standard deviation',  np.std(scores))

Test of the CNN
Scores for : n_blocks : 1 | filter_base : 32 | epsilon : 0.1 | lr : 0.1
Average Score 5.5
score standard deviation 2.247220505424423
Test of the CNN
Scores for : n_blocks : 1 | filter_base : 32 | epsilon : 0.1 | lr : 0.5
Average Score 5.1
score standard deviation 2.244994432064365
Test of the CNN
Scores for : n_blocks : 1 | filter_base : 32 | epsilon : 0.2 | lr : 0.1
Average Score 4.0
score standard deviation 2.345207879911715
Test of the CNN
Scores for : n_blocks : 1 | filter_base : 32 | epsilon : 0.2 | lr : 0.5
Average Score 3.9
score standard deviation 4.1400483088968905
Test of the CNN
Scores for : n_blocks : 1 | filter_base : 64 | epsilon : 0.1 | lr : 0.1
Average Score 1.75
score standard deviation 1.631716887208072
Test of the CNN
Scores for : n_blocks : 1 | filter_base : 64 | epsilon : 0.1 | lr : 0.5
Average Score 4.55
score standard deviation 3.304920573932148
Test of the CNN
Scores for : n_blocks : 1 | filter_base : 64 | epsilon : 0.2 | lr : 0.1
Average Score 4

In [63]:
epochs_train = 50
epochs_test = 10

memory_size = 2000
batch_size = 32


n_block = 1
filter_base = 32
epsilon = 0.1
lr = 0.1

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(n_blocks=n_block, filter_base=filter_base, lr=lr, grid_size=size, epsilon=epsilon, memory_size=memory_size, batch_size=batch_size)
train(agent, env, epochs_train, prefix='cnn_train')
HTML(display_videos('cnn_train20.mp4'))

print('Test of the CNN')
test(agent, env, epochs_test, prefix='cnn_test')

Epoch 000/050 | Loss 0.0698 | Win/lose count 4.0/4.0 (0.0)
Epoch 001/050 | Loss 0.0023 | Win/lose count 3.5/4.0 (-0.5)
Epoch 002/050 | Loss 0.0037 | Win/lose count 5.0/5.0 (0.0)
Epoch 003/050 | Loss 0.0175 | Win/lose count 8.5/13.0 (-4.5)
Epoch 004/050 | Loss 0.0129 | Win/lose count 4.5/5.0 (-0.5)
Epoch 005/050 | Loss 0.0036 | Win/lose count 8.0/4.0 (4.0)
Epoch 006/050 | Loss 0.0047 | Win/lose count 2.0/3.0 (-1.0)
Epoch 007/050 | Loss 0.0047 | Win/lose count 6.0/1.0 (5.0)
Epoch 008/050 | Loss 0.0052 | Win/lose count 5.5/1.0 (4.5)
Epoch 009/050 | Loss 0.0038 | Win/lose count 5.5/8.0 (-2.5)
Epoch 010/050 | Loss 0.0075 | Win/lose count 11.5/2.0 (9.5)
Epoch 011/050 | Loss 0.0127 | Win/lose count 10.5/6.0 (4.5)
Epoch 012/050 | Loss 0.0676 | Win/lose count 10.0/7.0 (3.0)
Epoch 013/050 | Loss 0.0104 | Win/lose count 11.0/13.0 (-2.0)
Epoch 014/050 | Loss 0.0053 | Win/lose count 11.0/4.0 (7.0)
Epoch 015/050 | Loss 0.0033 | Win/lose count 4.0/4.0 (0.0)
Epoch 016/050 | Loss 0.0170 | Win/lose coun

25.5

### Results

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [71]:
epochs_test = 15
for temperature in range(1,10):
  print('temperature is {}'.format(temperature/10))
  env = Environment(grid_size=size, max_time=T,temperature=temperature/10)
  agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
  agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

  agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
  agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
  print('Test of the CNN')
  test(agent_cnn,env,epochs_test,prefix='cnn_test')
  print('Test of the FC')
  test(agent_fc,env,epochs_test,prefix='fc_test')


temperature is 0.1
Test of the CNN
Win/lose count 3.0/2.0. Average score (1.0)
Win/lose count 1.5/2.0. Average score (0.25)
Win/lose count 1.5/0. Average score (0.6666666666666666)
Win/lose count 1.0/0. Average score (0.75)
Win/lose count 1.5/0. Average score (0.9)
Win/lose count 2.0/1.0. Average score (0.9166666666666666)
Win/lose count 2.0/1.0. Average score (0.9285714285714286)
Win/lose count 2.0/2.0. Average score (0.8125)
Win/lose count 0/0. Average score (0.7222222222222222)
Win/lose count 0/1.0. Average score (0.55)
Win/lose count 2.0/2.0. Average score (0.5)
Win/lose count 2.5/0. Average score (0.6666666666666666)
Win/lose count 0/2.0. Average score (0.46153846153846156)
Win/lose count 2.5/2.0. Average score (0.4642857142857143)
Win/lose count 2.0/1.0. Average score (0.5)
Final score: 0.5
Test of the FC
Win/lose count 2.0/1.0. Average score (1.0)
Win/lose count 1.5/1.0. Average score (0.75)
Win/lose count 0.5/1.0. Average score (0.3333333333333333)
Win/lose count 3.0/1.0. Avera

In [72]:
HTML(display_videos('cnn_test10.mp4'))

In [73]:
HTML(display_videos('fc_test10.mp4'))

The CNN architectures seems to give better resutls. However, the variance in the results is very high and the performance not so great.

We can observe that the algorithms don't explore the whole map and visit the same place very often (sometimes even oscillating between 2 positions). This can be explain by the fact that this method is a good solution to avoid decreasing the score. 

With an incresing temperature this phenomenon appears less and the score improves. 


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,decay_epsilon=0.1,prefix='', verbose=1):
    """ 
    This function train the agent with the env for a given number of epoch. The 
    random exploration paramater epsilon is decayed by a factor decay_epsilon at 
    each epoch
    """
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        agent.set_epsilon(agent.epsilon * (1 - decay_epsilon))
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train = True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        if verbose > 0:
          print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
                .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size, grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.3

        self.t = self.t + 1
        reward += self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state


In [56]:
# Training
epochs_train = 100
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, n_blocks=2, filter_base = 32, lr=0.1, epsilon = 0.9, memory_size=2000, batch_size = 32, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore', decay_epsilon=0.05)
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/100 | Loss 0.0154 | Win/lose count 8.5/53.29999999999985 (-44.79999999999985)
Epoch 001/100 | Loss 0.0172 | Win/lose count 7.0/54.899999999999835 (-47.899999999999835)
Epoch 002/100 | Loss 0.0051 | Win/lose count 14.5/52.099999999999866 (-37.599999999999866)
Epoch 003/100 | Loss 0.0184 | Win/lose count 13.5/53.79999999999985 (-40.29999999999985)
Epoch 004/100 | Loss 0.0112 | Win/lose count 10.5/52.69999999999987 (-42.19999999999987)
Epoch 005/100 | Loss 0.0124 | Win/lose count 11.0/53.79999999999985 (-42.79999999999985)
Epoch 006/100 | Loss 0.0133 | Win/lose count 10.0/53.299999999999855 (-43.299999999999855)
Epoch 007/100 | Loss 0.0057 | Win/lose count 10.0/58.59999999999989 (-48.59999999999989)
Epoch 008/100 | Loss 0.0124 | Win/lose count 13.0/52.19999999999989 (-39.19999999999989)
Epoch 009/100 | Loss 0.0593 | Win/lose count 16.5/49.99999999999994 (-33.49999999999994)
Epoch 010/100 | Loss 0.0190 | Win/lose count 9.5/54.19999999999987 (-44.69999999999987)
Epoch 011/100 | Lo

In [58]:
# Evaluation
epochs_test = 10
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 23.0/5.0. Average score (18.0)
Win/lose count 14.5/6.0. Average score (13.25)
Win/lose count 16.5/9.0. Average score (11.333333333333334)
Win/lose count 4.5/2.0. Average score (9.125)
Win/lose count 19.5/6.0. Average score (10.0)
Win/lose count 24.0/11.0. Average score (10.5)
Win/lose count 23.0/10.0. Average score (10.857142857142858)
Win/lose count 16.0/2.0. Average score (11.25)
Win/lose count 18.0/9.0. Average score (11.0)
Win/lose count 8.0/4.0. Average score (10.3)
Final score: 10.3


The exploring strategy seems to improve the performance of the agent which ends up being blocked only when cheese cells are far away. 


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***

In [0]:
class DQN_mime(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN_mime, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, a_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, a_]) # we only need the state and the action 

        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, a_  = self.memory.random_access()
            input_states[i] = s_
            target_q[i,a_] = 1
  
        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model


class DQN_CNN_mime(DQN_mime):
    def __init__(self, *args,n_blocks=2, filter_base=32, lr=0.1,**kwargs):
        super(DQN_CNN_mime, self).__init__(*args,**kwargs)
        
        input_model = keras.layers.Input(shape=(5, 5, self.n_state), name="input")
        x = input_model
        for n_block in range(n_blocks):
            x = Conv2D((2**n_block) * filter_base, 3, padding="same", activation="relu", name="conv{}_1".format(n_block))(x)
            x = Conv2D((2**n_block) * filter_base, 3, padding="same", activation="relu", name="conv{}_2".format(n_block))(x)
            x = MaxPooling2D(name="pooled_{}".format(n_block), strides=1)(x) # we are not downscaling 
        x = keras.layers.Flatten(name="flatten")(x)
        x = Dense(4, name="output")(x)
        model = keras.models.Model(input_model, x)

        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model


In [0]:
def train_mimick(agent,env,epoch,expert,prefix='', verbose=1):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = expert.act(state)
            
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state,action)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        if verbose > 0:
          print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
                .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [59]:
epochs_train = 75 
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
mime = DQN_CNN_mime(size, n_blocks=1, filter_base = 32, lr=0.1, epsilon = 0.5, memory_size=2000, batch_size = 32, n_state=3)
train_mimick(mime, env, epochs_train, agent)
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/075 | Loss 0.0046 | Win/lose count 3.5/3.0 (0.5)
Epoch 001/075 | Loss 0.0075 | Win/lose count 4.0/3.0 (1.0)
Epoch 002/075 | Loss 0.0053 | Win/lose count 7.5/2.0 (5.5)
Epoch 003/075 | Loss 0.0251 | Win/lose count 16.5/11.0 (5.5)
Epoch 004/075 | Loss 0.0295 | Win/lose count 18.0/9.0 (9.0)
Epoch 005/075 | Loss 0.0134 | Win/lose count 18.0/7.0 (11.0)
Epoch 006/075 | Loss 0.0211 | Win/lose count 8.5/3.0 (5.5)
Epoch 007/075 | Loss 0.0188 | Win/lose count 9.5/2.0 (7.5)
Epoch 008/075 | Loss 0.0103 | Win/lose count 13.5/1.0 (12.5)
Epoch 009/075 | Loss 0.0323 | Win/lose count 4.5/0 (4.5)
Epoch 010/075 | Loss 0.0253 | Win/lose count 14.0/7.0 (7.0)
Epoch 011/075 | Loss 0.0096 | Win/lose count 5.5/2.0 (3.5)
Epoch 012/075 | Loss 0.0248 | Win/lose count 19.0/7.0 (12.0)
Epoch 013/075 | Loss 0.0317 | Win/lose count 23.0/8.0 (15.0)
Epoch 014/075 | Loss 0.0191 | Win/lose count 6.5/1.0 (5.5)
Epoch 015/075 | Loss 0.0380 | Win/lose count 13.5/6.0 (7.5)
Epoch 016/075 | Loss 0.0325 | Win/lose count 

In [60]:
epochs_test = 10
test(mime,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 14.0/4.0. Average score (10.0)
Win/lose count 12.0/10.0. Average score (6.0)
Win/lose count 11.5/12.0. Average score (3.8333333333333335)
Win/lose count 16.0/9.0. Average score (4.625)
Win/lose count 22.5/9.0. Average score (6.4)
Win/lose count 9.5/9.0. Average score (5.416666666666667)
Win/lose count 15.5/16.0. Average score (4.571428571428571)
Win/lose count 20.0/12.0. Average score (5.0)
Win/lose count 23.0/16.0. Average score (5.222222222222222)
Win/lose count 11.5/11.0. Average score (4.75)
Final score: 4.75


The mime agent is performing better than the agent without exploration. However, it performs worse than the expert it has learnt from.
